https://www.sciencedirect.com/science/article/pii/S2211812814005318

In [7]:
# import dependencies
%matplotlib inline
#import keras
from keras.models import Sequential
from keras.activations import *
from keras.layers import *
from keras.losses import *
from keras.optimizers import *
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import talos as ta
import warnings
warnings.filterwarnings("ignore")

# suppres NumPy arrays scientific notation and round decimals to three places
np.set_printoptions(suppress=True)
np.printoptions(precision=3, suppress=True)

#libraries = [np,
#             pd,
#             sklearn,
#             keras,
#            ta]
#for library in libraries:
#    print(library, ' ', library.__version__)


requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

In [8]:
# import training data
train_data = pd.read_csv('Training_Features_and_Labels.txt', delim_whitespace=True, encoding='ISO-8859-1')
train_data.drop(columns=['Sr._No.'], inplace=True)
# import validation data
test_data = pd.read_csv('Test_Features_and_Labels.txt', delim_whitespace=True, encoding='ISO-8859-1')
test_data.drop(columns=['Sr._No.'], inplace=True)
# segregate a single row for validation
validation_data = test_data.loc[0]
test_data = test_data.iloc[1:]

In [9]:
# create NumPy arrays of training features and labels
train_features = train_data[['MRR(mm3/min)', 'Ra(µm)']].values
train_labels = train_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values
print('Training Features:\n\n', train_features)
print('\nTraining Labels:\n\n', train_labels) 

Training Features:

 [[ 306.67    0.36]
 [ 609.76    0.47]
 [ 909.28    0.52]
 [ 582.94    0.8 ]
 [1158.73    0.9 ]
 [1727.36    1.1 ]
 [ 943.34    1.63]
 [1875.96    1.76]
 [2797.84    2.17]
 [ 793.04    0.38]
 [1577.02    0.48]
 [2351.92    0.54]
 [1555.25    0.81]
 [3092.37    0.82]
 [4611.35    0.97]
 [2196.85    1.92]
 [4366.5     1.91]
 [6508.93    2.03]
 [ 875.13    0.29]
 [1735.95    0.37]
 [2582.46    0.38]
 [1745.24    0.82]
 [3461.88    0.79]
 [5149.9     1.11]
 [2549.2     1.75]
 [5055.49    1.89]
 [7518.86    1.82]]

Training Labels:

 [[ 280.        0.0508    0.4   ]
 [ 280.        0.0508    0.8   ]
 [ 280.        0.0508    1.2   ]
 [ 280.        0.1016    0.4   ]
 [ 280.        0.1016    0.8   ]
 [ 280.        0.1016    1.2   ]
 [ 280.        0.1524    0.4   ]
 [ 280.        0.1524    0.8   ]
 [ 280.        0.1524    1.2   ]
 [ 710.        0.0508    0.4   ]
 [ 710.        0.0508    0.8   ]
 [ 710.        0.0508    1.2   ]
 [ 710.        0.1016    0.4   ]
 [ 710.        0

In [10]:
# create NumPy arrays of validation features and labels
test_features = test_data[['MRR(mm3/min)', 'Ra(µm)']].values
test_labels = test_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values
print('Testing Features:\n\n', test_features)
print('\nTesting Labels:\n\n', test_labels)

Testing Features:

 [[2020.44    0.29]
 [6054.12    1.87]
 [5329.21    1.54]
 [5197.29    1.45]]

Testing Labels:

 [[1100.      0.05    0.69]
 [1095.      0.15    1.2 ]
 [1097.      0.13    1.14]
 [1094.      0.12    1.15]]


In [11]:
# create NumPy array for validation data
validation_features = validation_data[['MRR(mm3/min)', 'Ra(µm)']].values
validation_labels = validation_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values

# reshape the validation data because it's only one row
validation_features = validation_features.reshape(1, -1)
validation_labels = validation_labels.reshape(1, -1)

print('Validation Features:\n\n', validation_features)
print('\nValidation Labels:\n\n', validation_labels)

Validation Features:

 [[1106.51    0.21]]

Validation Labels:

 [[1100.      0.05    0.4 ]]


In [12]:
# Transforms features by scaling each feature to a given range, -1 and 1 in this case
MinMax_scaler = MinMaxScaler(feature_range=(-1, 1))
train_features_MinMax = MinMax_scaler.fit_transform(train_features)
train_labels_MinMax = MinMax_scaler.fit_transform(train_labels)
test_features_MinMax = MinMax_scaler.fit_transform(test_features)
test_labels_MinMax = MinMax_scaler.fit_transform(test_labels)
validation_features_MinMax = MinMax_scaler.fit_transform(validation_features)
validation_labels_MinMax = MinMax_scaler.fit_transform(validation_labels)

#MinMax_scaler.inverse_transform(validation_labels_MinMax)

In [13]:
# Standardize features by removing the mean and scaling to unit variance
Standard_scaler = StandardScaler()
train_features_Standard = Standard_scaler.fit_transform(train_features)
train_labels_Standard = Standard_scaler.fit_transform(train_labels)
test_features_Standard = Standard_scaler.fit_transform(test_features)
test_labels_Standard = Standard_scaler.fit_transform(test_labels)
validation_features_Standard = Standard_scaler.fit_transform(validation_features)
validation_labels_Standard = Standard_scaler.fit_transform(validation_labels)

# Standard_scaler.inverse_transform(train_features_Standard)

In [14]:
p = {
    'first_neuron': [8, 12, 24],
    'second_neuron': [36, 48, 62, 84],
    'third_neuron': [36, 48, 62, 84],
    'fourth_neuron': [8, 12, 24],
    'batch_size': [10, 20, 30],
    'activation': [relu,
                   #softmax,
                   #selu,
                   #softplus,
                   #softsign,
                   #tanh,
                   #sigmoid,
                   #hard_sigmoid,
                   #exponential,
                   linear],
    'optimizer' : [#'SGD',
                   #'RMSprop',
                   #'Adagrad',
                   #'Adadelta',
                   'Adam',],
                   #'Adamax',
                   #'Nadam',],
    'loss' : [mean_squared_error,]
              #mean_absolute_error,] 
              #mean_absolute_percentage_error, 
              #mean_squared_logarithmic_error,
              #squared_hinge,hinge,
              #categorical_hinge,
              #logcosh,],
}

In [15]:
def MinMax_ann(train_features, train_labels, test_features, test_labels, params):
    
    # replace the hyperparameter inputs with references to params dictionary 
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=2, activation=params['activation']))
    model.add(Dense(params['second_neuron'], activation=params['activation']))
    model.add(Dense(params['third_neuron'], activation=params['activation']))
    model.add(Dense(params['fourth_neuron'], activation=params['activation']))
    model.add(Dense(3, activation='linear'))
    model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])
    
    # make sure history object is returned by model.fit()
    out = model.fit(train_features_MinMax, train_labels_MinMax,
                    epochs=100,
                    batch_size=params['batch_size'],
                    verbose=0,
                    validation_data=[test_features_MinMax, test_labels_MinMax])
    
    # model output
    return out, model

In [16]:
def Standard_ann(train_features, train_labels, test_features, test_labels, params):
    
    # replace the hyperparameter inputs with references to params dictionary 
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=2, activation=params['activation']))
    model.add(Dense(params['second_neuron'], activation=params['activation']))
    model.add(Dense(params['third_neuron'], activation=params['activation']))
    model.add(Dense(params['fourth_neuron'], activation=params['activation']))
    model.add(Dense(3, activation='linear'))
    model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])
    
    # make sure history object is returned by model.fit()
    out = model.fit(train_features_Standard, train_labels_Standard,
                    epochs=100,
                    batch_size=params['batch_size'],
                    verbose=0,
                    validation_data=[test_features_Standard, test_labels_Standard])
    
    # model output
    return out, model

In [17]:
features_MinMax = np.concatenate((train_features_MinMax, test_features_MinMax), axis=0)
labels_MinMax = np.concatenate((train_labels_MinMax, test_labels_MinMax), axis=0)

features_Standard = np.concatenate((train_features_Standard, test_features_Standard), axis=0)
labels_Standard = np.concatenate((train_labels_Standard, test_labels_Standard), axis=0)

In [18]:
# run the Talos experiment on MinMax data
MinMax_t = ta.Scan(features_MinMax, labels_MinMax, 
            params=p, 
            model=MinMax_ann,
            )

  0%|                                                                                          | 0/864 [00:00<?, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W0820 11:59:33.641922  8964 deprecation_wrapper.py:119] From C:\Users\MacalusoC\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0820 11:59:33.683689  8964 deprecation_wrapper.py:119] From C:\Users\MacalusoC\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 11:59:33.700176  8964 deprecation_wrapper.py:119] From C:\Users\MacalusoC\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0820 11:59:33.866790  8964 deprecation_wrappe

In [19]:
# run the Talos experiment on Standard data
Standard_t = ta.Scan(features_Standard, labels_Standard, 
            params=p, 
            model=Standard_ann,
            )

100%|████████████████████████████████████████████████████████████████████████████████| 864/864 [43:32<00:00,  1.17s/it]


In [25]:
# use Scan object as input
r = ta.Reporting(MinMax_t)
# get the best paramaters
r.best_params()[0]

array([8, 'Adam', <function linear at 0x000001AA17118048>, 36, 10, 36, 8,
       0], dtype=object)

In [26]:
# use Scan object as input
r = ta.Reporting(Standard_t)
# get the best paramaters
r.best_params()[0]

array([12, 'Adam', <function relu at 0x000001AA17107D08>, 36, 20, 48, 8,
       0], dtype=object)

In [23]:
df = pd.read_csv('MinMax.csv')
df.head(3)

,round_epochs,val_loss,val_acc,loss,acc,activation,batch_size,first_neuron,fourth_neuron,loss.1,optimizer,second_neuron,third_neuron
0,100,0.261867,0.75,0.275643,0.481481,<function relu at 0x000001AA17107D08>,30,24,8,<function mean_squared_error at 0x000001AA170D...,Adam,48,48
1,100,0.277996,1.00,0.264963,0.518519,<function relu at 0x000001AA17107D08>,10,8,8,<function mean_squared_error at 0x000001AA170D...,Adam,36,84
2,100,0.302887,0.25,0.452316,0.555556,<function relu at 0x000001AA17107D08>,30,12,8,<function mean_squared_error at 0x000001AA170D...,Adam,48,48


In [27]:
# Define the MinMax model
MinMax_model = Sequential()
MinMax_model.add(Dense(8, input_dim=2, activation='relu'))
MinMax_model.add(Dense(36, activation='relu'))
MinMax_model.add(Dense(84, activation='relu'))
MinMax_model.add(Dense(8, activation='relu'))
MinMax_model.add(Dense(3, activation='linear'))
MinMax_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

MinMax_model.fit(train_features_MinMax,
                 train_labels_MinMax,
                 validation_data=(test_features_MinMax, test_labels_MinMax),
                 batch_size=10,
                 epochs=100)

Train on 27 samples, validate on 4 samples
Epoch 1/100
27/27 [==============================] - ETA: 0s - loss: 0.5252 - acc: 0.200 - 1s 20ms/step - loss: 0.6438 - acc: 0.2593 - val_loss: 0.6721 - val_acc: 0.2500
Epoch 2/100
27/27 [==============================] - ETA: 0s - loss: 0.6170 - acc: 0.200 - 0s 0us/step - loss: 0.6279 - acc: 0.3704 - val_loss: 0.6595 - val_acc: 0.2500
Epoch 3/100
27/27 [==============================] - ETA: 0s - loss: 0.5924 - acc: 0.800 - 0s 0us/step - loss: 0.6166 - acc: 0.6667 - val_loss: 0.6491 - val_acc: 0.2500
Epoch 4/100
27/27 [==============================] - ETA: 0s - loss: 0.6488 - acc: 0.400 - 0s 579us/step - loss: 0.6060 - acc: 0.5926 - val_loss: 0.6443 - val_acc: 0.2500
Epoch 5/100
27/27 [==============================] - ETA: 0s - loss: 0.5873 - acc: 0.500 - 0s 1ms/step - loss: 0.5973 - acc: 0.6296 - val_loss: 0.6392 - val_acc: 0.2500
Epoch 6/100
27/27 [==============================] - ETA: 0s - loss: 0.4805 - acc: 0.700 - 0s 577us/step - lo

Epoch 49/100
27/27 [==============================] - ETA: 0s - loss: 0.2611 - acc: 0.600 - 0s 579us/step - loss: 0.2630 - acc: 0.6296 - val_loss: 0.6808 - val_acc: 0.5000
Epoch 50/100
27/27 [==============================] - ETA: 0s - loss: 0.1822 - acc: 0.400 - 0s 579us/step - loss: 0.2626 - acc: 0.5926 - val_loss: 0.6768 - val_acc: 0.5000
Epoch 51/100
27/27 [==============================] - ETA: 0s - loss: 0.3071 - acc: 0.400 - 0s 579us/step - loss: 0.2633 - acc: 0.5926 - val_loss: 0.6678 - val_acc: 0.5000
Epoch 52/100
27/27 [==============================] - ETA: 0s - loss: 0.1316 - acc: 0.800 - 0s 579us/step - loss: 0.2613 - acc: 0.6296 - val_loss: 0.6690 - val_acc: 0.5000
Epoch 53/100
27/27 [==============================] - ETA: 0s - loss: 0.2955 - acc: 0.500 - 0s 0us/step - loss: 0.2607 - acc: 0.5926 - val_loss: 0.6611 - val_acc: 0.5000
Epoch 54/100
27/27 [==============================] - ETA: 0s - loss: 0.1268 - acc: 0.500 - 0s 579us/step - loss: 0.2612 - acc: 0.5926 - val_l

Epoch 97/100
27/27 [==============================] - ETA: 0s - loss: 0.1187 - acc: 0.700 - 0s 579us/step - loss: 0.2438 - acc: 0.5556 - val_loss: 0.5926 - val_acc: 0.5000
Epoch 98/100
27/27 [==============================] - ETA: 0s - loss: 0.1674 - acc: 0.700 - 0s 0us/step - loss: 0.2430 - acc: 0.5556 - val_loss: 0.5892 - val_acc: 0.5000
Epoch 99/100
27/27 [==============================] - ETA: 0s - loss: 0.1740 - acc: 0.600 - 0s 0us/step - loss: 0.2440 - acc: 0.5556 - val_loss: 0.5872 - val_acc: 0.5000
Epoch 100/100
27/27 [==============================] - ETA: 0s - loss: 0.2436 - acc: 0.500 - 0s 579us/step - loss: 0.2428 - acc: 0.5556 - val_loss: 0.5986 - val_acc: 0.5000


In [24]:
df = pd.read_csv('Standard.csv')
df.head(3)

,round_epochs,val_loss,val_acc,loss,acc,activation,batch_size,first_neuron,fourth_neuron,loss.1,optimizer,second_neuron,third_neuron
0,100,0.386672,0.75,0.459910,0.666667,<function relu at 0x000001AA17107D08>,30,24,8,<function mean_squared_error at 0x000001AA170D...,Adam,48,62
1,100,0.466409,0.75,0.401966,0.777778,<function relu at 0x000001AA17107D08>,10,12,8,<function mean_squared_error at 0x000001AA170D...,Adam,84,84
2,100,0.511823,0.75,0.447284,0.629630,<function linear at 0x000001AA17118048>,20,24,12,<function mean_squared_error at 0x000001AA170D...,Adam,36,62


# Define the Standard model
Standard_model = Sequential()
Standard_model.add(Dense(24, input_dim=2, activation='relu'))
Standard_model.add(Dense(84, activation='relu'))
Standard_model.add(Dense(84, activation='relu'))
Standard_model.add(Dense(12, activation='relu'))
Standard_model.add(Dense(3, activation='linear'))
Standard_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

Standard_model.fit(train_features_Standard,
                   train_labels_Standard,
                   validation_data=(test_features_Standard, test_labels_Standard),
                   batch_size=20,
                   epochs=100)

In [28]:
# MinMax prediction
prediction = MinMax_model.predict(validation_features_MinMax)
prediction = MinMax_scaler.inverse_transform(prediction)
# compare
MinMax_scaler.inverse_transform(validation_labels_MinMax)
print('MinMax Prediction:\n\n', prediction)
print('\nActual:\n\n', MinMax_scaler.inverse_transform(validation_labels_MinMax))

MinMax Prediction:

 [[1100.296       -0.0317607    0.5831349]]

Actual:

 [[1100.      0.05    0.4 ]]


# Standard prediction
prediction = Standard_model.predict(validation_features_Standard)
prediction = Standard_scaler.inverse_transform(prediction)
# compare
print('Standard Prediction:\n\n', prediction)
print('\nActual:\n\n', Standard_scaler.inverse_transform(validation_labels_Standard))